In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pickle
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

training_data_path = '/content/drive/MyDrive/data'

Mounted at /content/drive


#HEIC to JPG

In [ ]:
# Install pillow-heif to handle HEIC files
!pip install pillow-heif

# Convert all HEIC to JPG
from PIL import Image
from pillow_heif import register_heif_opener
import os

register_heif_opener()  # Enable HEIC support

converted = 0
for root, dirs, files in os.walk(training_data_path):
    for file in files:
        if file.lower().endswith('.heic'):
            heic_path = os.path.join(root, file)
            jpg_path = heic_path.rsplit('.', 1)[0] + '.jpg'

            try:
                img = Image.open(heic_path)
                img.convert('RGB').save(jpg_path, 'JPEG')
                os.remove(heic_path)  # Delete original HEIC
                converted += 1
                print(f"Converted: {file}")
            except Exception as e:
                print(f"Failed: {file} - {e}")

print(f"\nTotal converted: {converted} HEIC files")


Total converted: 0 HEIC files


In [ ]:
# Copy dataset to Colab local storage (fast)
import shutil

print("Copying dataset to local storage...")
local_path = '/content/dataset'

if os.path.exists(local_path):
    shutil.rmtree(local_path)

shutil.copytree(training_data_path, local_path)
print(f"Dataset copied to {local_path}")

# Update path
training_data_path = local_path

Copying dataset to local storage...
✓ Dataset copied to /content/dataset


In [7]:
batch_size = 128

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

# No augmentation for validation
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    training_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
test_generator = val_datagen.flow_from_directory(
    training_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")
print(f"Training images: {train_generator.samples}")
print(f"Validation images: {test_generator.samples}")

Found 1424 images belonging to 91 classes.
Found 340 images belonging to 91 classes.
Number of classes: 91
Training images: 1424
Validation images: 340


# Person Name Mapping

In [8]:
results_map = {v: k for k, v in train_generator.class_indices.items()}

with open('results_map.pkl', 'wb') as f:
    pickle.dump(results_map, f)

#Model

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#Freezed Training

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 152s 12s/step - accuracy: 0.0719 - loss: 4.5157 - val_accuracy: 0.3214 - val_loss: 3.3403
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 105s 10s/step - accuracy: 0.2562 - loss: 3.4205 - val_accuracy: 0.4935 - val_loss: 2.5995
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 105s 10s/step - accuracy: 0.3755 - loss: 2.7554 - val_accuracy: 0.6169 - val_loss: 1.9433
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 103s 9s/step - accuracy: 0.4715 - loss: 2.2770 - val_accuracy: 0.7500 - val_loss: 1.5204
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 103s 9s/step - accuracy: 0.6019 - loss: 1.7049 - val_accuracy: 0.8052 - val_loss: 1.1685
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 106s 10s/step - accuracy: 0.6431 - loss: 1.5704 - val_accuracy: 0.8279 - val_loss: 0.9608
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 103s 9s/step - accuracy: 0.7451 - loss: 1.1823 - val_accuracy: 0.8409 - val_loss: 0.7929
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 104s 10s/step - accuracy: 0.7691 - loss: 1.0282 - val_accuracy: 0.870

# Unfreezed Training

In [ ]:
base_model.trainable = True


for layer in base_model.layers[:50]:
    layer.trainable = False


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


fine_tune_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5
)

Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 185s 12s/step - accuracy: 0.7049 - loss: 1.2081 - val_accuracy: 0.7305 - val_loss: 0.9801
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 104s 9s/step - accuracy: 0.8690 - loss: 0.5286 - val_accuracy: 0.7500 - val_loss: 0.8603
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 102s 9s/step - accuracy: 0.9098 - loss: 0.3823 - val_accuracy: 0.8214 - val_loss: 0.5846
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 103s 9s/step - accuracy: 0.9282 - loss: 0.3105 - val_accuracy: 0.8929 - val_loss: 0.3669
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 104s 9s/step - accuracy: 0.9485 - loss: 0.2269 - val_accuracy: 0.9221 - val_loss: 0.2883


# Evals

In [ ]:
final_loss, final_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {final_accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 5s/step - accuracy: 0.9161 - loss: 0.2907
Test accuracy: 92.21%


# Save Model

In [10]:
model.save('face_recognition_transfer_learning.keras')
model.save('face_recognition_transfer_learning.h5')

print("Model saved")

Model saved


In [11]:
# Save model properly
model.save('face_recognition_transfer_learning.keras', save_format='keras')

# Verify it loads
test_model = tf.keras.models.load_model('face_recognition_transfer_learning.keras')


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
